In [ ]:
import numpy as np
import geopandas as gpd
import contextily as cx
import matplotlib.pyplot as plt
import shapely.geometry as sg
import pathlib
import shutil
from tqdm.notebook import tqdm

import set_path
from gvl.helper_functions import (roundup, rounddown, box_to_name,
                                  get_tilecode_from_filename)

In [ ]:
dataset_path = '../datasets/validation/'

RD_CRS = 'epsg:28992'
LL_CRS = 'WGS84'

In [ ]:
area = gpd.read_file(f'{dataset_path}correct_trees_areas_oosterpark.gpkg', crs=RD_CRS)
trees = gpd.read_file(f'{dataset_path}correct_trees_oosterpark.gpkg', crs=LL_CRS)

In [ ]:
trees.to_crs(RD_CRS, inplace=True)

In [ ]:
%matplotlib widget

fig, ax = plt.subplots(1, figsize=(10,7))

area.plot(ax=ax, color='lightblue', alpha=0.8)
trees.plot(ax=ax, color='darkgreen', marker='.', markersize=5, alpha=1)
cx.add_basemap(ax=ax, crs=RD_CRS)

## Compare with extracted trees

In [ ]:
import pickle

with open('oosterpark_trees.pickle', 'rb') as handle:
    data = pickle.load(handle)

extracted = gpd.GeoDataFrame(data=data,
                             geometry='concave_hull',
                             crs='epsg:28992')

In [ ]:
extr_match = extracted[extracted.intersects(area.unary_union)]

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,7))

extr_match.plot(ax=ax, color='green', alpha=0.8)
area.plot(ax=ax, color='lightblue', alpha=0.8)
trees.plot(ax=ax, color='black', marker='.', markersize=7, alpha=1)
extr_match.set_geometry('location').plot(ax=ax, color='red', alpha=1, marker='.', markersize=7)

plt.show()

In [ ]:
cover = extr_match.unary_union

In [ ]:
trees['recalled'] = trees.apply(lambda row: cover.contains(row.geometry), axis=1)

In [ ]:
extr_match['precise'] = extr_match.apply(lambda row: trees.within(row['convex_hull']).any(), axis=1)

In [ ]:
# Total number of ground truth trees
print(f'Total #trees in validation area: {len(trees)}')
# Total number of extracted trees
print(f'Total #trees extracted: {len(extr_match)}')
# Precision
print(f"Precision: {sum(extr_match['precise']) / len(extr_match) * 100:.1f}")
# Recall
print(f"Recall: {sum(trees['recalled']) / len(trees) * 100:.1f}")

## File copying

In [ ]:
# Get bounds of city area
bounds = area.unary_union.bounds

tile_size = 1000

In [ ]:
BASE_FOLDER = '/media/dbloembergen/PointCloud/AHN4'
ahn_subtile_folder = f'{BASE_FOLDER}/AMS_subtiles_1000/'

In [ ]:
subtiles = list(pathlib.Path(ahn_subtile_folder).glob('*.laz'))
subtiles_codes = [get_tilecode_from_filename(subtile.name) for subtile in subtiles]

In [ ]:
def code_to_geom(code):
    parts = code.split('_')
    x, y = int(parts[0])*tile_size, int(parts[1])*tile_size
    return sg.box(x, y, x+tile_size, y+tile_size)

geoms = [code_to_geom(code) for code in subtiles_codes]

In [ ]:
subtiles_gdf = gpd.GeoDataFrame({'subtile': subtiles,
                                 'name': subtiles_codes,
                                 'geometry': geoms}, crs=RD_CRS)

In [ ]:
subtiles_gdf = subtiles_gdf[subtiles_gdf.intersects(area.unary_union)]

In [ ]:
subtiles_gdf

In [ ]:
subtiles_gdf.name.values

In [ ]:
out_folder = '../datasets/AHN4/AMS_subtiles_1000/'
dry_run = True

laz_files = subtiles_gdf.subtile.values
npz_files = [f'{BASE_FOLDER}/npz_subtiles_1000/ahn_{name}.npz' for name in subtiles_gdf.name.values]

for f in tqdm(laz_files):
    if dry_run:
        print(f'{f.as_posix()} => {out_folder}{f.name}')
    else:
        shutil.copy(f.as_posix(), f'{out_folder}{f.name}')